In [2]:
import pandas as pd
import numpy as np

In [47]:


def process_metric_tables(metric_tables):
    def parse_tables_manual(metric_tables):
        dataframes = []
        for table in metric_tables:
            lines = table.strip().split('\n')
            headers = lines[0].split(maxsplit=2)
            data = []
            for line in lines[1:]:
                index_space = line.index(' ')
                first_split = line[:index_space].strip()
                remaining = line[index_space:].strip()
                parts = remaining.rsplit(maxsplit=2)
                if len(parts) == 3:
                    data.append([parts[0], parts[1], parts[2]])
                else:
                    data.append([parts[0], np.nan, np.nan])
            df = pd.DataFrame(data, columns=headers)
            df[headers[1]] = pd.to_numeric(df[headers[1]], errors='coerce')
            df[headers[2]] = pd.to_numeric(df[headers[2]], errors='coerce')
            dataframes.append(df)
        return dataframes

    def calculate_statistics_ordered(dataframes):
        metrics = dataframes[0]['Metric']
        result_list = []
        for metric in metrics:
            metric_data = [df[df['Metric'] == metric] for df in dataframes]
            metric_stats = {}
            for col in ['Training', 'Validation']:
                valid_data = pd.concat([md[col] for md in metric_data]).dropna()
                if not valid_data.empty:
                    mean_val = valid_data.mean()
                    std_val = valid_data.std()
                    metric_stats[col] = (mean_val, std_val)
                else:
                    metric_stats[col] = np.nan
            result_list.append((metric, metric_stats))
        return result_list

    def format_output_ordered(statistics_ordered):
        output_df = pd.DataFrame(columns=['Metric', 'Training', 'Validation'])
        for metric, values in statistics_ordered:
            row = {'Metric': metric}
            for col in ['Training', 'Validation']:
                if col in values and not pd.isna(values[col]):
                    row[col] = f"({values[col][0]:.3f}, {values[col][1]:.3f})"
                else:
                    row[col] = np.nan
            output_df = pd.concat([output_df, pd.DataFrame([row])], ignore_index=True)
        output_string = output_df.to_string(index=False)
        return output_string

    # Process the input tables
    dataframes = parse_tables_manual(metric_tables)
    statistics = calculate_statistics_ordered(dataframes)
    output_table = format_output_ordered(statistics)
    return output_table
# Example usage
metric_tables = [
    """                              Metric  Training  Validation
0                   Average Accuracy     0.359       0.388
1     Standard Deviation of Accuracy     0.032       0.067
2                      Best Accuracy     0.427       0.523
3                      Last Accuracy     0.426       0.518
4                        Overall AUC       NaN       0.505
5                   Maximum F1 Score       NaN       0.198
6                       Minimum Loss       NaN       1.020
7  Difference in Average Loss Last N       NaN      -0.020
8  Standard Deviation of Loss Last N       NaN       0.025""",


"""                              Metric  Training  Validation
0                   Average Accuracy     0.835       0.710
1     Standard Deviation of Accuracy     0.179       0.118
2                      Best Accuracy     0.979       0.789
3                      Last Accuracy     0.974       0.759
4                        Overall AUC       NaN       0.525
5                   Maximum F1 Score       NaN       0.791
6                       Minimum Loss       NaN       0.754
7  Difference in Average Loss Last N       NaN       1.198
8  Standard Deviation of Loss Last N       NaN       0.306""",


"""                              Metric  Training  Validation
0                   Average Accuracy     0.349       0.342
1     Standard Deviation of Accuracy     0.017       0.018
2                      Best Accuracy     0.401       0.379
3                      Last Accuracy     0.349       0.374
4                        Overall AUC       NaN       0.466
5                   Maximum F1 Score       NaN       0.000
6                       Minimum Loss       NaN       1.083
7  Difference in Average Loss Last N       NaN      -0.001
8  Standard Deviation of Loss Last N       NaN       0.003""", 

"""                              Metric  Training  Validation
0                   Average Accuracy     0.636       0.579
1     Standard Deviation of Accuracy     0.251       0.207
2                      Best Accuracy     0.941       0.824
3                      Last Accuracy     0.941       0.779
4                        Overall AUC       NaN       0.501
5                   Maximum F1 Score       NaN       0.823
6                       Minimum Loss       NaN       0.883
7  Difference in Average Loss Last N       NaN       0.879
8  Standard Deviation of Loss Last N       NaN       0.219""",

"""                              Metric  Training  Validation
0                   Average Accuracy     0.809       0.683
1     Standard Deviation of Accuracy     0.199       0.122
2                      Best Accuracy     0.970       0.773
3                      Last Accuracy     0.866       0.621
4                        Overall AUC       NaN       0.489
5                   Maximum F1 Score       NaN       0.765
6                       Minimum Loss       NaN       0.891
7  Difference in Average Loss Last N       NaN       1.643
8  Standard Deviation of Loss Last N       NaN       0.426"""



    # More tables would follow here
]

result = process_metric_tables(metric_tables)
print(result)


                           Metric       Training     Validation
                 Average Accuracy (0.598, 0.235) (0.540, 0.168)
   Standard Deviation of Accuracy (0.136, 0.105) (0.106, 0.070)
                    Best Accuracy (0.744, 0.301) (0.658, 0.196)
                    Last Accuracy (0.711, 0.299) (0.610, 0.170)
                      Overall AUC            NaN (0.497, 0.022)
                 Maximum F1 Score            NaN (0.515, 0.387)
                     Minimum Loss            NaN (0.926, 0.129)
Difference in Average Loss Last N            NaN (0.740, 0.737)
Standard Deviation of Loss Last N            NaN (0.196, 0.182)
